<a href="https://colab.research.google.com/github/anupampani123/text-generator1/blob/main/textgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#LSTM model for text generation 
#Text generation based on text by nietzche 

import tensorflow as tf 
import numpy as np
path = tf.keras.utils.get_file( 'nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt') 
text = open(path).read().lower()
print('Corpus length:', len(text))

614400/600901 [==============================] - 1s 2us/step
Corpus length: 600893


In [2]:
#Steps : 1) extract a fixed length of overlapping characters 
#        2) one-hot encode them (basically the target)
#        3) Input will then be a 3D array (Sequences, maxlen, unique_characters) with corresponding target output 

maxlen=60
step=3
sentences =[] #this holds the sequences
next_chars=[] #Holds the follow up characters - e.g this is a ca_ , so it will hold t 

for i in range(0,len(text)-maxlen,step):    #increment by step 3 : this then his(space), then is(space)a etc.
  sentences.append(text[i:i+maxlen]);
  next_chars.append(text[i+maxlen]);

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))   #To get unique characters 

char_indices = dict((char, chars.index(char)) for char in chars) #dictionary to map unique characters to their respective index in text

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)    # Input will then be a 3D array (Sequences, maxlen, unique_characters)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)    
#One hot encoding         
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1




Number of sequences: 200278


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [3]:
#Building the LSTM network 
#Simple network - LSTM followed by a dense layer followed by softmax

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(maxlen, len(chars)))) 
model.add(tf.keras.layers.Dense(len(chars), activation='softmax'))

In [4]:
#Since one hot encoding is used, we will use categorical cross-entropy
#Read about the parameters again later
optimizer = tf.keras.optimizers.RMSprop(lr=0.01) 
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [5]:
# from the model, the probability distribution of the next character is drawn using the available text 
# reweight the distribution using the softmax temperature 
# sample next character according to new distribution 
# Add the character to the end of the available text 

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:

import random
import sys

# model.fit(x, y, batch_size=128, epochs=60)
# start_index = random.randint(0, len(text) - maxlen - 1) 
# generated_text = text[start_index: start_index + maxlen] 
# print('--- Generating with seed: "' + generated_text + '"')


for epoch in range(1, 20):
    print('epoch', epoch)
    
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(70):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


epoch 1
1565/1565 [==============================] - 17s 7ms/step - loss: 1.9680
--- Generating with seed: "n, and often think how i can
still further advance him, and "
------ temperature: 0.2
n, and often think how i can
still further advance him, and the one for the result to prosesses of the say the expecian of the one
------ temperature: 0.5
r the result to prosesses of the say the expecian of the one more that the instincted of the late the
essentic in to the graties f
------ temperature: 1.0
 the instincted of the late the
essentic in to the graties flow of mesterpoatian aracts and hadily is hais of the
essean of contam
------ temperature: 1.2
terpoatian aracts and hadily is hais of the
essean of contame of much as to eutice knowc), it the ordon protectiouquously., the ph
epoch 2
1565/1565 [==============================] - 11s 7ms/step - loss: 1.6164
--- Generating with seed: "and certain in our experiences, that man is the rule, nature"
------ temperature: 0.2
and certain in o